In [100]:
import torch
from train import train_model
from dataset.utils import *
from utils import *
import numpy as np

In [101]:
batch_size = 32
def redefine_train_parameters():
    global device, epochs, learning_rate, loss_lr, num_classes, embedding_size
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    epochs = 2
    learning_rate = 0.001
    loss_lr = 0.01
    num_classes = get_num_classes(dataset)
    embedding_size = 512

In [102]:
def validate(train_dataset, test_dataset, title=False):
    global r1,r2,r4,r8,r16,r32
    train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
    test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
    model = train_model(model_name='resnet50',
                    loss_func_name='proxy_anchor',
                    num_classes=num_classes,
                    dataloader=train_loader,
                    epochs=epochs,
                    learning_rate=learning_rate,
                    verbose=0)
    if title:
        print(f'Recall {title}:')
    r1,r2,r4,r8,r16,r32 = evaluate_cos(model, test_loader)
    return r1,r2,r4,r8,r16,r32

In [103]:
def cross_validate(dataset, k=10):
    global r1,r2,r4,r8,r16,r32
    k=k
    split_len = len(dataset)//k
    max_index = split_len*k
    r1,r2,r4,r8,r16,r32= [],[],[],[],[],[]
    for fold in range(k):
        print(f'''----------
fold {fold+1}:''')
        start = split_len * fold
        train_dataset = dataset[:start] + dataset[start + split_len:max_index]
        test_dataset = dataset[start:start+split_len]
        train_loader = load_dataset(train_dataset, batch_size, shuffle=True)
        test_loader = load_dataset(test_dataset, batch_size, shuffle=False)
        model = train_model(model_name='resnet50',
                        loss_func_name='proxy_anchor',
                        num_classes=num_classes,
                        dataloader=train_loader,
                        epochs=epochs,
                        learning_rate=learning_rate,
                        verbose=0)
        recall = evaluate_cos(model, test_loader)
        r1.append(recall[0])
        r2.append(recall[1])
        r4.append(recall[2])
        r8.append(recall[3])
        r16.append(recall[4])
        r32.append(recall[5])

In [104]:
def avg_recall(name):
    print(f'''Average Recall {name}:
R@1 : {np.mean(r1)*100:.4f}
R@2 : {np.mean(r2)*100:.4f}
R@4 : {np.mean(r4)*100:.4f}
R@8 : {np.mean(r8)*100:.4f}
R@16 : {np.mean(r16)*100:.4f}
R@32 : {np.mean(r32)*100:.4f}''')

In [67]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [68]:
validate(train_dataset,test_dataset, title='PA')

Finished Training, Time: 331.2395 Best loss: 8.7518


100%|██████████| 262/262 [04:02<00:00,  1.08it/s]


R@1 : 1.670
R@2 : 2.350
R@4 : 3.209
R@8 : 4.306
R@16 : 5.905
R@32 : 7.921


(0.016700465227245616,
 0.023499940355481332,
 0.032088751043779076,
 0.0430633424788262,
 0.059048073482047,
 0.07920792079207921)

In [70]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=0.2)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100)
redefine_train_parameters()

In [71]:
validate(train_dataset,test_dataset,title='PA+Flip(.2)')

Finished Training, Time: 331.7281 Best loss: 8.7108


100%|██████████| 262/262 [04:03<00:00,  1.08it/s]


R@1 : 0.549
R@2 : 1.038
R@4 : 1.765
R@8 : 2.911
R@16 : 4.509
R@32 : 6.585


(0.005487295717523559,
 0.010378146248359775,
 0.017654777525945367,
 0.029106525110342358,
 0.04509125611356316,
 0.06584754861028272)

In [73]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v')
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v')
redefine_train_parameters()

In [74]:
validate(train_dataset,test_dataset,title='PA+AdaptiveV')

Finished Training, Time: 422.8876 Best loss: 8.7061


100%|██████████| 262/262 [05:10<00:00,  1.18s/it]


R@1 : 1.062
R@2 : 1.706
R@4 : 2.481
R@8 : 3.579
R@16 : 5.237
R@32 : 7.193


(0.010616724323034713,
 0.017058332339258023,
 0.024812119766193488,
 0.035786711201240606,
 0.052367887391148754,
 0.07193128951449362)

In [76]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median=True)
redefine_train_parameters()

In [77]:
validate(train_dataset,test_dataset,title='PA+Median')

Finished Training, Time: 436.5227 Best loss: 8.7455


100%|██████████| 262/262 [05:20<00:00,  1.22s/it]


R@1 : 0.680
R@2 : 1.121
R@4 : 2.099
R@8 : 3.292
R@16 : 4.986
R@32 : 7.324


(0.006799475128235715,
 0.011213169509722056,
 0.02099487057139449,
 0.03292377430514136,
 0.04986281760706191,
 0.07324346892520578)

In [79]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, hist='v', median_before=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, hist='v', median_before=True)
redefine_train_parameters()

In [80]:
validate(train_dataset,test_dataset,title='PA+Median+AdaptiveV')

Finished Training, Time: 525.8643 Best loss: 8.6879


100%|██████████| 262/262 [06:27<00:00,  1.48s/it]


R@1 : 0.966
R@2 : 1.598
R@4 : 2.445
R@8 : 3.614
R@16 : 5.392
R@32 : 7.873


(0.009662412024334963,
 0.015984731003220804,
 0.02445425265418108,
 0.03614457831325301,
 0.053918644876535846,
 0.07873076464272934)

In [82]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True)
redefine_train_parameters()

In [83]:
validate(train_dataset,test_dataset,title='PA+Unsharp')

/opt/conda/lib/python3.8/site-packages/skimage/filters/_unsharp_mask.py:11: FutureWarning: Automatic detection of the color channel was deprecated in v0.19, and `channel_axis=None` will be the new default in v0.22. Set `channel_axis=-1` explicitly to silence this warning.
  blurred = gaussian(image, sigma=radius, mode='reflect')


Finished Training, Time: 379.6614 Best loss: 8.7570


100%|██████████| 262/262 [04:37<00:00,  1.06s/it]


R@1 : 2.493
R@2 : 4.020
R@4 : 5.809
R@8 : 7.897
R@16 : 11.130
R@32 : 13.933


(0.024931408803530956,
 0.040200405582726945,
 0.05809376118334725,
 0.07896934271740427,
 0.11129667183585829,
 0.13932959561016342)

In [105]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [106]:
validate(train_dataset,test_dataset,title='PA+MedianUnsharp+Median')

Finished Training, Time: 497.0166 Best loss: 8.7641
Recall PA+MedianUnsharp+Median:


100%|██████████| 262/262 [06:06<00:00,  1.40s/it]


R@1 : 0.930
R@2 : 1.682
R@4 : 2.827
R@8 : 4.175
R@16 : 5.762
R@32 : 7.646


(0.009304544912322557,
 0.016819754264583084,
 0.028271501848980078,
 0.04175116306811404,
 0.057616605033997376,
 0.07646427293331742)

In [86]:
validate(train_dataset,test_dataset,title='PA+MedianUnsharp+Median')

Finished Training, Time: 497.2281 Best loss: 8.7409


100%|██████████| 262/262 [06:05<00:00,  1.40s/it]


R@1 : 0.799
R@2 : 1.121
R@4 : 1.909
R@8 : 2.935
R@16 : 4.652
R@32 : 6.549


(0.007992365501610402,
 0.011213169509722056,
 0.01908624597399499,
 0.029345103185017297,
 0.046522724561612785,
 0.06548968149827031)

In [107]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, flip=.2, median_unsharp=True, median=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, median_unsharp=True, median=True)
redefine_train_parameters()

In [108]:
validate(train_dataset,test_dataset,title='PA+Flip(.2)+MedianUnsharp+Median')

Finished Training, Time: 498.9320 Best loss: 8.7180
Recall PA+Flip(.2)+MedianUnsharp+Median:


100%|██████████| 262/262 [06:06<00:00,  1.40s/it]


R@1 : 0.763
R@2 : 1.264
R@4 : 1.956
R@8 : 2.923
R@16 : 4.485
R@32 : 6.322


(0.007634498389597996,
 0.012644637957771682,
 0.019563402123344866,
 0.02922581414767983,
 0.044852678038888226,
 0.06322318978885841)

In [89]:
validate(train_dataset,test_dataset,title='PA+Flip(.2)+MedianUnsharp+Median')

Finished Training, Time: 497.0486 Best loss: 8.7220


100%|██████████| 262/262 [06:05<00:00,  1.39s/it]


R@1 : 2.422
R@2 : 3.805
R@4 : 5.666
R@8 : 7.456
R@16 : 9.519
R@32 : 12.263


(0.024215674579506144,
 0.03805320291065251,
 0.056662292735297624,
 0.07455564833591793,
 0.09519265179530001,
 0.12262913038291781)

In [91]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, adjust_log=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, adjust_log=True)
redefine_train_parameters()

In [92]:
validate(train_dataset,test_dataset,title='PA+AdjustLog')

Finished Training, Time: 335.7468 Best loss: 8.7906


100%|██████████| 262/262 [04:05<00:00,  1.07it/s]


R@1 : 1.587
R@2 : 2.445
R@4 : 3.662
R@8 : 5.392
R@16 : 7.432
R@32 : 9.829


(0.015865441965883336,
 0.02445425265418108,
 0.03662173446260289,
 0.053918644876535846,
 0.074317070261243,
 0.09829416676607419)

In [94]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, unsharp=True, adjust_log=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, unsharp=True, adjust_log=True)
redefine_train_parameters()

In [95]:
validate(train_dataset,test_dataset,title='PA+Unsharp+AdjustLog')

Finished Training, Time: 384.5573 Best loss: 8.7529


100%|██████████| 262/262 [04:40<00:00,  1.07s/it]


R@1 : 1.157
R@2 : 1.813
R@4 : 2.779
R@8 : 3.805
R@16 : 5.535
R@32 : 7.790


(0.011571036621734462,
 0.01813193367529524,
 0.027794345699630203,
 0.03805320291065251,
 0.05535011332458547,
 0.07789574138136705)

In [97]:
train_dataset = make_dataset('LFWCustom', split='train', image_size=100, adjust_log=True, unsharp_after=True)
test_dataset = make_dataset('LFWCustom', split='test', image_size=100, adjust_log=True, unsharp_after=True)
redefine_train_parameters()

In [98]:
validate(train_dataset,test_dataset,title='PA+AdjustLog+Unsharp')

Finished Training, Time: 383.5622 Best loss: 8.7977


100%|██████████| 262/262 [04:40<00:00,  1.07s/it]


R@1 : 1.384
R@2 : 2.219
R@4 : 2.970
R@8 : 4.247
R@16 : 5.619
R@32 : 7.790


(0.013837528331146367,
 0.022187760944769177,
 0.0297029702970297,
 0.04246689729213885,
 0.05618513658594775,
 0.07789574138136705)